In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [47]:
data = pd.read_csv('exp_data.csv')
survey = pd.read_csv('exit_survey.csv')

In [48]:
data.head()

,NAME,CONDITION (P/NP),"CONDITION (NM,M/ M,NM)",Unnamed: 3,ASSIGNED GENRE,TIME 1st LAP,TIME 2nd LAP
0,Saloni Agrawal,P,"NM, M",NaN,Hip-Hop,2:11,2:10
1,Sreejith Santhosh,P,"M, NM",NaN,Hip-Hop,1:52,1:41
2,Aries Cole,P,"NM, M",NaN,Hip-Hop,1:45,1:51
3,Baden Gloyer,NP,"M, NM",NaN,Pop,2:33,2:40
4,Andy Ho,P,"M, NM",NaN,Pop,1:57,1:59


In [49]:
survey.head()

,name,ex_first,ex_second
0,Saloni Agrawal,5,4
1,Sreejith Santhosh,5,4
2,Baden Gloyer,5,5
3,Jordi Pham,3,6
4,Nitika Suresh,8,8


In [55]:
data = pd.read_csv('exp_data.csv')
#drop empty col
data = data.drop(columns=['Unnamed: 3'])
# rename columns to something easiere
cols = ['name', 'cond_pref', 'cond_order', 'genre', 'time_first', 'time_second']
data.columns = cols
# change preference and music first lap into booleans
# data = data.assign(pref_bool=(data['cond_pref'] == 'P')).assign(music_first_bool=(data['cond_order']=='M, NM'))
data = data.assign(pref_bool=(data['cond_pref'] == 'P'))
data['first_cond'] = data['cond_order'].apply(lambda x: 'Music' if x[0]=='M' else 'No Music')
# get relevant columns
data = data[['name', 'pref_bool', 'first_cond', 'genre', 'time_first', 'time_second']]

data.head()

,name,pref_bool,first_cond,genre,time_first,time_second
0,Saloni Agrawal,True,No Music,Hip-Hop,2:11,2:10
1,Sreejith Santhosh,True,Music,Hip-Hop,1:52,1:41
2,Aries Cole,True,No Music,Hip-Hop,1:45,1:51
3,Baden Gloyer,False,Music,Pop,2:33,2:40
4,Andy Ho,True,Music,Pop,1:57,1:59


In [56]:
def time_to_seconds(time_str):
    # Split the time string by ":"
    minutes, seconds = map(int, time_str.split(':'))
    # Convert to total seconds
    total_seconds = minutes * 60 + seconds
    return total_seconds

In [57]:
# convert times into seconds
data['time_first'] = data['time_first'].transform(time_to_seconds)
data['time_second'] = data['time_second'].transform(time_to_seconds)

data.head()

,name,pref_bool,first_cond,genre,time_first,time_second
0,Saloni Agrawal,True,No Music,Hip-Hop,131,130
1,Sreejith Santhosh,True,Music,Hip-Hop,112,101
2,Aries Cole,True,No Music,Hip-Hop,105,111
3,Baden Gloyer,False,Music,Pop,153,160
4,Andy Ho,True,Music,Pop,117,119


In [58]:
# merge times and participant experience
all_data = data.merge(survey, on='name')
all_data.head()

,name,pref_bool,first_cond,genre,time_first,time_second,ex_first,ex_second
0,Saloni Agrawal,True,No Music,Hip-Hop,131,130,5,4
1,Aries Cole,True,No Music,Hip-Hop,105,111,7,4
2,Baden Gloyer,False,Music,Pop,153,160,5,5
3,Andy Ho,True,Music,Pop,117,119,9,10
4,Jordi Pham,False,Music,Classical,94,89,3,6


In [63]:
(all_data
 .groupby(['pref_bool', 'first_cond'])[['time_first', 'time_second']]
 .mean().diff(axis=1))

time_first  time_second
pref_bool first_cond                         
False     Music              NaN     2.714286
          No Music           NaN     4.750000
True      Music              NaN     2.500000
          No Music           NaN     2.666667